In [1]:
# Click on Runtime/Run all 
#
# Example program for AI generating sample text for College Essay
# The model trains on college essay for 20 cycle and  generates the output based on seed text.
# You can change the seed text to see how AI predicts the output for different seed text.

<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/TensorFlow%20In%20Practice/Course%203%20-%20NLP/Course%203%20-%20Week%204%20-%20Lesson%202%20-%20Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [3]:
# Reading in teh test file of essays
!wget --no-check-certificate \
    https://drive.google.com/file/d/1605gzszfVtGADmd1r1FJtNJQiknzVOd8/view?usp=sharing \
    -O /tmp/essay10.txt

--2020-08-31 01:23:57--  https://drive.google.com/file/d/1605gzszfVtGADmd1r1FJtNJQiknzVOd8/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 74.125.142.102, 74.125.142.138, 74.125.142.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/tmp/essay10.txt’

/tmp/essay10.txt        [ <=>                ]  69.59K  --.-KB/s    in 0.02s   

2020-08-31 01:23:58 (2.79 MB/s) - ‘/tmp/essay10.txt’ saved [71260]



In [4]:
tokenizer = Tokenizer()

#data = open('/tmp/irish-lyrics-eof.txt').read()
#data = open('/tmp/paradiselost.txt').read()
#data = open('/content/modernessay.txt').read()
data = open('/tmp/essay10.txt').read()
corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

#print(tokenizer.word_index)
#print(total_words)


In [5]:
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=20, verbose=1)
#print model.summary()
print(model)


Epoch 1/20
434/434 [==============================] - 179s 412ms/step - loss: 5.1153 - accuracy: 0.1407
Epoch 2/20
434/434 [==============================] - 179s 413ms/step - loss: 3.6770 - accuracy: 0.2636
Epoch 3/20
434/434 [==============================] - 179s 412ms/step - loss: 2.9584 - accuracy: 0.3372
Epoch 4/20
434/434 [==============================] - 180s 414ms/step - loss: 2.4812 - accuracy: 0.3961
Epoch 5/20
434/434 [==============================] - 180s 416ms/step - loss: 2.1047 - accuracy: 0.4552
Epoch 6/20
252/434 [================>.............] - ETA: 1:15 - loss: 1.6963 - accuracy: 0.5239

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')


In [ ]:
#seed_text = "Life is a balance"
#seed_text = "Life is a challange"
#seed_text = "Curiosity"
seed_text = "I like computer science"

next_words = 60
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = model.predict_classes(token_list, verbose=0)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print("---------------------")	
print("---------------------")	
print(seed_text)